In [1]:
# import librart
import sklearn
import numpy
import pandas as pd
import pickle
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy
from scipy.sparse import csr_matrix
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import feature_select
import feture_process
from scipy.sparse import load_npz
from scipy.sparse import save_npz
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [3]:
# import data
train_df = pd.read_csv("./project_data_files/book_rating_train.csv")
test_df = pd.read_csv("./project_data_files/book_rating_test.csv")


One hot code Processing

In [4]:
# process Prublisher and Language
train_df["Language"], test_df["Language"] = feture_process.docclass_preprocess(train_df["Language"],test_df["Language"],10)
train_df["Publisher"], test_df["Publisher"] = feture_process.docclass_preprocess(train_df["Publisher"],test_df["Publisher"],200)

In [6]:
# use oneHotCode for Publisher (sklearn)
publisher_train_hot = feture_process.process_OneHotEncoder(train_df,"Publisher")
publisher_test_hot = feture_process.process_OneHotEncoder(test_df,"Publisher")
train_df = pd.concat([train_df, publisher_train_hot], axis=1)
test_df = pd.concat([test_df, publisher_test_hot], axis=1)

In [7]:
# use oneHotCode for Language (sklearn)
language_train_hot = feture_process.process_OneHotEncoder(train_df,"Language")
language_test_hot = feture_process.process_OneHotEncoder(test_df,"Language")
train_df = pd.concat([train_df, language_train_hot], axis=1)
test_df = pd.concat([test_df, language_test_hot], axis=1)


In [ ]:
# use oneHotCode for Publisher (pandas)
publisher_train_hot,publisher_test_hot = feture_process.process_OneHotEncoder_pd(train_df,test_df,"Publisher")
train_df = pd.concat([train_df, publisher_train_hot], axis=1)
test_df = pd.concat([test_df, publisher_test_hot], axis=1)

In [ ]:
# use oneHotCode for Language (pandas)
language_train_hot,language_test_hot = feture_process.process_OneHotEncoder_pd(train_df,test_df,"Language")
train_df = pd.concat([train_df, language_train_hot], axis=1)
test_df = pd.concat([test_df, language_test_hot], axis=1)

Countvectorizer processing

In [8]:
# train_name_countvectorizer
train_name_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_name_countvectorizer.pkl", "rb"))
train_name_dic = train_name_countvectorizer.vocabulary_

# train_authors_countvectorizer
train_authors_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_authors_countvectorizer.pkl", "rb"))
train_authors_dic = train_authors_countvectorizer.vocabulary_

# train_desc_countvectorizer
train_desc_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_desc_countvectorizer.pkl", "rb"))
train_desc__dic = train_desc_countvectorizer.vocabulary_

# process vector features
train_name_features = train_name_countvectorizer.transform(train_df['Name'])
train_authors_features = train_authors_countvectorizer.transform(train_df['Authors'])
train_desc_features = train_desc_countvectorizer.transform(train_df['Description'])
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language', 'rating_label'])


In [9]:
# new train features
sparse_features = hstack([train_name_features, train_authors_features, train_desc_features])
dense_features = csr_matrix(other_features_df.values)
train_features = hstack([sparse_features, dense_features])


# process test features
test_name_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_name_vec.npz')
test_authors_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_authors_vec.npz')
test_desc_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_desc_vec.npz')
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])

test_sparse_features = hstack([test_name_features, test_authors_features, test_desc_features])
new_dense_features = csr_matrix(test_other_features_df.values)
test_features = hstack([test_sparse_features, new_dense_features])

doc2vec processing

In [ ]:
# use doc2vec
# use doc2vec
# use doc2vec
# use doc2vec

# process vector features
train_name_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_name_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
train_authors_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_authors_doc2vec20.csv", index_col = False, delimiter = ',', header=None)
train_desc_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_desc_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language', 'rating_label'])
train_features = pd.concat([train_name_features, train_authors_features, train_desc_features, other_features_df], axis=1)


In [ ]:
# use doc2vec
# use doc2vec
# use doc2vec
# use doc2vec

# process test features
test_name_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_name_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
test_authors_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_authors_doc2vec20.csv", index_col = False, delimiter = ',', header=None)
test_desc_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_desc_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])
test_features = pd.concat([test_name_features, test_authors_features, test_desc_features, test_other_features_df], axis=1)



Feature Selection

In [15]:
# mutualInformation selection
selected_features,selected_features_test = feature_select.MI(train_df,train_features,test_features,12000)

In [ ]:
# chi2
selected_features,selected_features_test = feature_select.chi_square(train_df,train_features,test_features,12000)

In [ ]:
# save npz
save_npz('selected_features.npz', selected_features)
save_npz('selected_features_test.npz', selected_features_test)

In [ ]:
# feature loading
selected_features = load_npz("...")
selected_features_test = load_npz("...")

Train model

-Decision Tree as clf

In [ ]:

X_train, X_val, y_train, y_val = train_test_split(selected_features, train_df["rating_label"], test_size=0.2, random_state=66)
# bagging 
base_learner = DecisionTreeClassifier()
bagging_clf = BaggingClassifier(base_estimator=base_learner, n_estimators=500, random_state=66)
bagging_clf.fit(X_train, y_train)



-Random Forest as clf

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(selected_features, train_df["rating_label"], test_size=0.2, random_state=66)
# bagging 
base_learner = RandomForestClassifier()
bagging_clf = BaggingClassifier(base_estimator=base_learner, n_estimators=500, random_state=66)
bagging_clf.fit(X_train, y_train)


Modle evalutaion

In [ ]:
accuracy = bagging_clf.score(X_val, y_val)
print("Accuracy:", accuracy)

In [ ]:
# export prediction
y_pred = bagging_clf.predict(selected_features_test)
output_df = pd.DataFrame({'rating_label': y_pred})
output_df.index += 1
output_df.index.name = 'id'
output_df.to_csv('./predictions.csv')